<a href="https://colab.research.google.com/github/MatheusMuriel/Testes-CrewAI/blob/main/TestesCrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U crewai crewai_tools

In [41]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["GEMINI_MODEL_NAME"] = 'gemini/gemini-1.5-flash'

os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [42]:
from crewai import Agent, Task, Crew, LLM

my_llm = LLM(
  model=os.environ["GEMINI_MODEL_NAME"],
  api_key=os.environ["GEMINI_API_KEY"]
)

In [30]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
#https://docs.crewai.com/concepts/tools#available-crewai-tools

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

#### Agentes  
https://docs.crewai.com/concepts/agents#agent-attributes  
1. Buscador  
2. Redator  
3. Editor

In [43]:
buscador = Agent(
  role='Buscador de conteúdo',
  goal='Busque conteúdo online sobre o tema {tema}',
  backstory='Você esta trabalhando na criação de artigos para o linkedin sobre {tema}.'
            'Você vai fazer uma busca sobre informações na internet, coletá-las e agrupa-lás.'
            'Seu trabalho servirá de base para o Redator de Conteúdo.',
  tools=[search_tool, scrape_tool],
  llm=my_llm,
  verbose=True
)

In [44]:
redator = Agent(
  role='Redator de conteúdo',
  goal='Escreva um texto de Linkedin sobre {tema}',
  backstory='Você esta trabalhando na criação de artigos para o linkedin sobre {tema}.'
            'Você vai utilizar os dados coletados pelo Buscador de Conteúdo para escrever um texto.'
            'Interessante, divertido e factualmente correto para o Linkedin.'
            'Dê opiniões sobre o {tema} mas ao faze-lo, deixe claro que são opiniões pessoais.',
  tools=[search_tool, scrape_tool],
  llm=my_llm,
  verbose=True
)

In [45]:
editor = Agent(
  role='Editor de conteúdo',
  goal='Editar um texto de Linkedin para que ele tenha um tom mais informal.',
  backstory='Você esta trabalhando na criação de artigos para o linkedin sobre {tema}.'
            'Você vai receber um texto do Redator de Conteúdo e edita-lo para um tom de voz'
            'mais informal e divertido.',
  tools=[search_tool, scrape_tool],
  llm=my_llm,
  verbose=True
)

#### Tarefas  
https://docs.crewai.com/concepts/tasks  


In [46]:
buscar = Task(
  description=(
    "1. Priorize as ultimas tendencias, os principais atores e as noticias mais relevantes sobre {tema}.\n"
    "2. Identifique o publico-alvo, considerando seus interesses e pontos de dor.\n"
    "3. Inclua palavras-chave de SEO e dados ou fontes relevantes."
  ),
  agent=buscador,
  expected_output="Um plano de tendencias sobre {tema}, com as palavras mais relevantes de SEO e as ultimas noticias."
)

In [47]:
redigir = Task(
  description=(
    "1. Use os dados coletados de conteudo para criar um post de Linkedin atraente sobre {tema}.\n"
    "2. Incorpore palavras-chave de SEO de forma natural.\n"
    "3. Certifique-se de que o post esteja estruturado dee forma cativante, com uma conlusão que faça o leitor refletir.\n"
  ),
  agent=redator,
  expected_output="Um texto de Linkedin sobre o tema {tema}."
)

In [48]:
editar = Task(
  description=(
    "Revisar a postafem do Linkledin em questão quanto a erros gramaticais e alinhamento com a voz de uma pessoa."
  ),
  agent=editor,
  expected_output='Um texto de Linkedin pronto para publicação, seguindo o tom de voz esperado.'
                  'O texto está separado em parágrafos e não usa bullet point.'
)

#### Equipe  
https://docs.crewai.com/concepts/crews  


In [49]:
equipe = Crew(
  agents=[buscador, redator, editor],
  tasks=[buscar, redigir, editar],
  verbose=True
)

#### Executando
Rodando o Crew


In [50]:
tema_do_artigo = 'Alimentação saudavel'
entradas = {"tema": tema_do_artigo}

#equipe.run(inputs=entradas)
resultado = equipe.kickoff(inputs=entradas)

# Agent: Buscador de conteúdo
## Task: 1. Priorize as ultimas tendencias, os principais atores e as noticias mais relevantes sobre Alimentação saudavel.
2. Identifique o publico-alvo, considerando seus interesses e pontos de dor.
3. Inclua palavras-chave de SEO e dados ou fontes relevantes.


# Agent: Buscador de conteúdo
## Thought: ```tool_code
Thought: I need to search for information about current trends, key players, and news in healthy eating.  I'll also need to identify the target audience and relevant keywords.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"healthy eating trends 2024\"}"
## Tool Output: 
{'searchParameters': {'q': 'healthy eating trends 2024', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Alarming trends call for action to define the future role of food in ...', 'link': 'https://newsroom.heart.org/news/alarming-trends-call-for-action-to-define-the-future-role-of-food-in-nations-health', 'snippet':

In [60]:
from IPython.display import Markdown
output = resultado.raw
output = output.replace("#", "\#")
Markdown(output)

Alimentação Saudável: Tendências, Mitos e Verdades (e a Minha Opinião!)

E aí, pessoal!  Cuidar da saúde virou prioridade, né? E a alimentação saudável é a estrela principal dessa história! Mas com tanta informação (e desinformação!) por aí, como a gente se encontra nesse universo de dietas da moda e promessas milagrosas?

Resolvi trazer algumas tendências de alimentação saudável para 2024, botar um pouco de ordem na casa e compartilhar minha humilde opinião (que, claro, não substitui o seu nutri!).

Em 2024, alguns destaques são:

Alimentos anti-inflamatórios:  A gente já sabe que inflamação crônica não é boa companhia.  Então, frutas vermelhas e vegetais coloridos, cheios de antioxidantes, são nossos novos melhores amigos!

Proteínas vegetais: Tofu, leguminosas... delícia e sustentável!  A onda plant-based tá só começando!

Microbiota intestinal:  Sabe aquela história de que a nossa saúde começa no intestino?  É verdade! Probióticos e prebióticos são essenciais pra uma flora feliz.

Alimentos funcionais: Além de nutritivos, alguns alimentos oferecem benefícios extras.  Adaptogênicos, por exemplo, são super interessantes!

Sustentabilidade:  Consumir alimentos locais e orgânicos é um super "win-win": bom pro planeta e pra nossa saúde!

Personalização:  Não existe fórmula mágica!  Um plano alimentar personalizado, que leve em conta as suas necessidades, é o caminho das pedras.

Mitos e Verdades (minha visão, claro!):

(Aqui você pode inserir exemplos de mitos e verdades sobre alimentação saudável, baseando-se em pesquisas e sua opinião. Por exemplo, o jejum intermitente pode ser legal, mas precisa de acompanhamento profissional!)

Minha Opinião Final (sem rodeios!):

Alimentação saudável não é dieta da moda, é estilo de vida! Busque informações confiáveis, priorize comida de verdade e não tenha medo de experimentar. Encontre o que funciona pra você e lembre-se: equilíbrio é tudo!

E você, qual a sua relação com a alimentação saudável? Conta aqui nos comentários! 👇

\#alimentacaosaudavel \#bemestar \#saude \#nutricao \#vidasaudavel \#alimentacaosustentavel \#dieta \#fitness \#receitassaudaveis \#bemviver